# Assignment: Preparing Data for Analysis (Modified Titanic)

![](https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/%20files/hw.png)

In [1]:
#Import the libraries
import numpy as np
import pandas as pd

In [49]:
df = pd.read_csv('https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/hw/titanic_training_dataset_v2.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0,PC 17599,71.2833,C85,C
2,3,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0,113803,53.1000,C123,S
4,5,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0,373450,8.0500,NaN,S


In [50]:
df["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [53]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder(handle_unknown='ignore')  # this feature will be all zeros

nominal_columns = ["Embarked"]
enc_df = pd.DataFrame(enc.fit_transform(df[nominal_columns]).toarray())

# Get category labels from the OneHotEncoder
categories = enc.categories_[0]
print("cat", categories)
# Set column names to enc_df based on category labels
enc_df.columns = [f"Embarked_{category}" for category in categories]

print(enc_df)


cat ['C' 'Q' 'S' nan]
     Embarked_C  Embarked_Q  Embarked_S  Embarked_nan
0           0.0         0.0         1.0           0.0
1           1.0         0.0         0.0           0.0
2           0.0         0.0         1.0           0.0
3           0.0         0.0         1.0           0.0
4           0.0         0.0         1.0           0.0
..          ...         ...         ...           ...
886         0.0         0.0         1.0           0.0
887         0.0         0.0         1.0           0.0
888         0.0         0.0         1.0           0.0
889         1.0         0.0         0.0           0.0
890         0.0         1.0         0.0           0.0

[891 rows x 4 columns]


# 1) Load data & review the data

<font color='blue'>Q1) What is the shape of this dataset? (rows & columns)</font>

In [5]:
print('rows:', df.shape[0])
print('columns:', df.shape[1])

rows: 891
columns: 12


# 2) Drop unqualified variables

*   Drop variables with missing > 50%
*   Drop categorical variables with flat values > 70% (variables with the same value in the same column)

<font color='blue'>Q2) How many columns do we have left?</font>




In [6]:
df = df.dropna(thresh=(0.5) * len(df), axis=1)

columns_to_drop = []
flat_threshold = 0.7

for column in df.columns:
    unique_values_percentage = df[column].value_counts().max() / len(df[column])
    if unique_values_percentage > flat_threshold:
        columns_to_drop.append(column)

df = df.drop(columns=columns_to_drop)
df.shape[1]

10

# 3) Remove all rows with missing target (the variable "Survived")

<font color='blue'>Q3) How many rows do we have left?</font>

In [7]:
df = df.dropna(subset = ['Survived'])
df.shape[0]

865

# 4) Handle outliers 

For the variable “Fare”, replace outlier values with the boundary values


*   If value < (Q1 - 1.5IQR), relace with (Q1 - 1.5IQR)
*   If value > (Q3 + 1.5IQR), relace with (Q3 + 1.5IQR)

<font color='blue'>Q4) What is the mean of “Fare” after replacing the outliers (round 2 decimal points)?</font>



In [8]:
Q1 = df['Fare'].quantile(0.25)
Q3 = df['Fare'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df['Fare'].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x)).mean()


24.038996878612718

# 5) Impute missing value



*   Show the number of missing values in each variable
*   Impute missing values with mean & mode
*   Show the number of missing values again after missing value imputation

<font color='blue'>Q5) Which variable has the largest number of missing values?</font>



In [9]:
from sklearn.impute import SimpleImputer

print("before impute missing values\n")
print(df.isnull().sum())

# Create a SimpleImputer object
imputer = SimpleImputer(strategy='mean')  # You can also use 'most_frequent' for mode

# Convert DataFrame to numpy array
df_num = df[['Pclass', 'Age', 'SibSp']]

# Convert the imputed numpy array back to a DataFrame
df[['Pclass', 'Age', 'SibSp']] = pd.DataFrame(imputer.fit_transform(df_num))

imputer = SimpleImputer(strategy='most_frequent')
df_cat = df[[ 'Name', 'Ticket', 'Embarked' ]]
df[[ 'Name', 'Ticket', 'Embarked' ]] = pd.DataFrame(imputer.fit_transform(df_cat))
df = df.dropna()

print("\nafter impute missing values\n")
print(df.isnull().sum())

before impute missing values

PassengerId      0
Survived         0
Pclass          63
Name            25
Sex              0
Age            172
SibSp           33
Ticket          39
Fare             0
Embarked        87
dtype: int64

after impute missing values

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [10]:
print("before impute missing values\n")
print(df.isnull().sum())

df["Pclass"] = df["Pclass"].fillna(df["Pclass"].mean())
df["Age"] = df["Age"].fillna(df["Age"].mean())
df["SibSp"] = df["SibSp"].fillna(df["SibSp"].mean())
df["Ticket"] = df["Ticket"].fillna(df["Ticket"].mode())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode())

categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)

print("\nafter impute missing values\n")
print(df.isnull().sum())

before impute missing values

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

after impute missing values

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


# 6) Convert categorical to numeric values



*   Drop the variables “Name” & “Ticket”
*   For the variables “Sex” & “Embarked”, perform the dummy coding and drop the first level. Also, drop those original variables (“Sex” & “Embarked”)

<font color='blue'>Q6) How many columns do we have?</font>



In [11]:
df = df.drop(["Name", "Ticket"], axis=1)
df = pd.get_dummies(df, columns=["Sex", "Embarked"], drop_first=True)
df.shape[1]


9

# 7) Partition data



*   Split train/test split with stratification using 70%:30% and random seed with 12345
*   Show a proportion between survived (1) and died (0) in all data sets (total data, train, test)

<font color='blue'>Q7) What is a proportion between survived (1) and died (0) in the training data?</font>



In [12]:
from sklearn.model_selection import train_test_split

# df2 = df.dropna(subset = ['Survived'])
# df2 = df.fillna(df.mean())

# categorical_columns = df2.select_dtypes(include=['object']).columns
# for column in categorical_columns:
#     mode_value = df2[column].mode()[0]
#     df2[column].fillna(mode_value, inplace=True)

X = df.drop('Survived', axis=1)
y = df['Survived']

# Perform train/test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=12345)

# Display proportions in all datasets
def display_proportions(data, label):
    proportion = data.value_counts(normalize=True)
    print(f"Proportion in {label} dataset:")
    print(proportion)
    print()

# Proportions in the training dataset
display_proportions(pd.DataFrame(y_train, columns=['Survived']), "Train")

# Proportions in the testing dataset
display_proportions(pd.DataFrame(y_test, columns=['Survived']), "Test")

print('rows of training data', y_train.shape[0])
print('rows of testing data', y_test.shape[0])

Proportion in Train dataset:
Survived
0.0         0.616695
1.0         0.383305
Name: proportion, dtype: float64

Proportion in Test dataset:
Survived
0.0         0.615079
1.0         0.384921
Name: proportion, dtype: float64

rows of training data 587
rows of testing data 252
